In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import json
import operator

from functools import reduce
from gensim.models import Word2Vec
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_distances

In [2]:
df = pd.read_csv("../Dataset/dataset.csv")
df = df.drop(columns="data")
df.head()

,author,lyrics,style,title
0,Midian Lima,"Amanheceu,Nada pesquei,Parecia ser apenas mais...",eletrônica,Não Pare
1,MC Doni,"Não adianta tu se declarar,Romance, compromiss...",eletrônica,Te Amo Sem Compromisso (To Nem Aí)
2,Sebastián Yatra,"I remember when I met you,I didn't wanna fall,...",eletrônica,My Only One (No Hay Nadie Más) (part. Isabela ...
3,Billie Eilish,"Thought I found a way,Thought I found a way ou...",eletrônica,Lovely (feat. Khalid)
4,Ed Sheeran,"I found a love for me,Darling, just dive right...",eletrônica,Perfect


## Reducing for 5 genres for runtime purpose

In [3]:
_id = reduce(
    operator.add,
    [
        df["style"] == "pop",
        df["style"] == "rock",
        df["style"] == "reggae",
        df["style"] == "indie",
        df["style"] == "country"
    ]
)
df = df.loc[_id]
df["style"].unique()

/Users/paulocarneiro/MusicClassifier/venv/lib/python3.7/site-packages/pandas/core/computation/expressions.py:194: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  op=op_str, alt_op=unsupported[op_str]


array(['pop', 'country', 'reggae', 'rock', 'indie'], dtype=object)

In [4]:
def clean_string(sentence):
    if not isinstance(sentence, str):
        return ""
    for p in string.punctuation:
        sentence = sentence.replace(p, " ")
    sentence = sentence.replace("  ", " ")
    return sentence.lower()

In [5]:
data = df
tokens = []
for index, row in data.iterrows():
    tokens.append(nltk.word_tokenize(clean_string(row["lyrics"])) +  nltk.word_tokenize(row["title"]))
model = Word2Vec(tokens, min_count=1)
print(len(model.wv.vocab))

35497


In [6]:
X = model[model.wv.vocab]

/Users/paulocarneiro/MusicClassifier/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [7]:
agglomerative = AgglomerativeClustering(
    n_clusters=None, affinity="cosine", linkage="complete", distance_threshold=0.3
)
agglomerative.fit_predict(X)

array([1207, 7516, 6577, ..., 2871, 7616,  472])

## Saving the clusters in a json file

In [8]:
words = list(model.wv.vocab)
new_array = np.asarray(agglomerative.labels_)
clusters = {}
clusters["clusters"] = {}
clusters["replace"] = {}
NUM_CLUSTERS = np.max(new_array)

for i in range(NUM_CLUSTERS):
    ids_ = np.where(new_array == i)
    print(f"=========== Cluster {i} ========== ")
    for ids in ids_[0]:
        if i not in clusters["replace"]:
            clusters["replace"][i] = words[ids]
        print(words[ids])
        clusters["clusters"][words[ids]] = i
        
with open("agglomerative_clusters.json", "w") as f:
    json.dump(clusters,f)

=========== Cluster 0 ========== 
Close
Honey
Remember
Sure
Cheatin
Really
Built
Pose
=========== Cluster 1 ========== 
concha
delato
tubo
folhetinho
imaginin
=========== Cluster 2 ========== 
astrais
brindo
selada
=========== Cluster 3 ========== 
caráter
bichano
=========== Cluster 4 ========== 
Stephen
Grazon
Post
Malone
RKCB
Marina
Problems
=========== Cluster 5 ========== 
crave
forgot
offer
=========== Cluster 6 ========== 
committed
speaking
quicksand
eternally
emblem
opportunities
Places
nonviolent
evermore
frankstein
scient
perseverant
Heads
unforecasted
=========== Cluster 7 ========== 
seat
bottom
hill
table
crown
bottle
page
swing
truck
glass
rose
=========== Cluster 8 ========== 
certains
disent
fazê
=========== Cluster 9 ========== 
poker
fishnet
sauce
firefight
braids
=========== Cluster 10 ========== 
Indecente
experimentar
aliás
adulto
Exhalt
acuse
desdizer
lembrado
Perdoa
=========== Cluster 11 ========== 
speechless
electrified
leavin
sociopath
=========== Cluster 12

Knows
5x
salems
enganação
Avô
groundhog
bullfrog
marie
dangers
toils
snares
versos
direitos
500
Roxanne
oferecida
viram
Luau
virei
dependente
poucas
pelos
negros
Bop
genealógico
Ones
ará
caçador
infrator
caminhão
morrem
decadence
negation
maliciously
dormem
atearam
Trench
deixadas
industry
séculos
lutando
assistem
Urbana
sirene
zählt
laut
zehn
Iridescent
exterminio
ska
Clearly
parlaments
distrai
atrai
seafoam
freezerburn
seagulls
agonyzing
pesadelos
tatunobilis
campari
azuis
Misery
Business
esperem
unites
ie
garantida
liberem
Skanking
Minuto
deletin
invenção
Lanterna
primatas
anota
imediata
bacu
oats
alphabetical
Meus
Teeth
Misread
arlésienne
Hanging
alchemy
Chocolate
Gore
gunning
shunning
vulto
cães
judiadas
=========== Cluster 175 ========== 
yeesh
shops
=========== Cluster 176 ========== 
campeira
medico
paulistão
turista
=========== Cluster 177 ========== 
things
deception
=========== Cluster 178 ========== 
physical
knack
grocery
karaoke
comedy
errors
=========== Cluster 179 =====

=========== Cluster 283 ========== 
twinny
bolder
=========== Cluster 284 ========== 
recetó
cuerpito
escribirles
pereza
duermo
hoja
blanco
=========== Cluster 285 ========== 
eating
Beautiful
errbody
kanye
mcgrath
abduction
Wanted
unanswered
Same
incompetent
combat
Lucky
contemplating
restraints
stricken
​
=========== Cluster 286 ========== 
singing
top
playing
=========== Cluster 287 ========== 
favela
mão
pele
janela
ideia
maluca
bunda
batida
escuridão
preferida
palavra
sala
roda
chamando
afiada
pilha
garupa
frequência
Wannabe
tirando
estrada
passada
ponta
pedra
escravos
estais
=========== Cluster 288 ========== 
selfish
curb
muscle
crossword
integers
=========== Cluster 289 ========== 
lordy
Malibu
=========== Cluster 290 ========== 
neither
gravy
biscuits
Biscuits
Stuck
=========== Cluster 291 ========== 
comprehend
replace
=========== Cluster 292 ========== 
abraçar
haunting
precedes
sócio
importar
ari
chew
nittos
Cornerstone
tarado
largoa
urinar
atirar
aventurar
kazoo
Ruby
=====

=========== Cluster 455 ========== 
trained
Aces
upfront
desmaiar
=========== Cluster 456 ========== 
entertain
fairy
pining
tape
scenes
tale
lite
waterfalls
brassiere
emptiness
untangled
Message
budapest
=========== Cluster 457 ========== 
Cigarettes
overweight
fluctuates
persistance
sift
=========== Cluster 458 ========== 
claro
mostrar
preciso
sou
faço
gosto
diferente
tava
estava
diria
vejo
nesse
acredito
meto
dizia
mandei
desço
tiver
acorda
tivesse
aprendi
balanço
feriu
ouça
doutor
lago
escondendo
mudaria
=========== Cluster 459 ========== 
fuerte
viaje
perro
vienes
faltan
último
mitad
brazos
pensé
ningún
hombre
tenemos
condena
enamorarme
atado
niña
entonces
intento
estrellas
llegarías
aimer
hommes
pies
quita
sábanas
cannabis
paris
ojitos
toas
sufriendo
acharner
ilusiones
presumas
telefono
litigando
libero
sanno
hipnotiza
cambiándome
tranqui
ana
doutait
paciencia
voltemo
infelizes
piedras
laberintos
embargo
fueron
=========== Cluster 460 ========== 
resist
unlove
bribe
=========== 

=========== Cluster 579 ========== 
1
Mess
Almost
tumba
gospel
Mullet
Stroll
piano
Without
Juice
WRLD
Sharaya
Nicky
Jam
Colby
O'Donis
dimension
Astral
Gabriela
Mioto
Carson
Pijama
DOLLAR
louisiana
elogiou
BloodPop
redecórame
Sue
Railway
bluejay
Summer
Zhavia
Snoop
Desperately
Horses
Should
Wash
Spanish
Living
Aguilera
Z
fbi
Pabllo
Vittar
milsap
Sober
Hymn
Lived
Outside
Trace
Dada
Jah
Poeta
Monster
Helicopter
jocoso
Blaze
Fat
Bottomed
Final
Clean
Departure
'Cause
Moved
Chair
Gods
=========== Cluster 580 ========== 
calar
diga
deixe
critique
apaixonei
desculpe
machucar
chame
ofereceram
culpar
=========== Cluster 581 ========== 
fiel
eran
betta
estudiar
blackjack
juntei
chris
Unwound
xadrez
Even
starin
redone
Just
Said
aliens
kim
marmalade
moulin
coucher
sah
Marmalade
credits
larga
pleurer
déjà
tanggalin
pagdududa
ahwoo
cobraras
supieran
cette
trestle
spanning
demonstrou
siguió
Roses
Set
Highwayman
estuve
fives
twenties
thourough
skinhead
deadhead
Dive
estranhoque
Before
chaque
brahma
vis

...

=========== Cluster 7614 ========== 
agosto
=========== Cluster 7615 ========== 
finja
=========== Cluster 7616 ========== 
academy
=========== Cluster 7617 ========== 
floated
=========== Cluster 7618 ========== 
ease
sweat
=========== Cluster 7619 ========== 
pastinha
=========== Cluster 7620 ========== 
livestream
=========== Cluster 7621 ========== 
noddin
=========== Cluster 7622 ========== 
unbearable
=========== Cluster 7623 ========== 
unplugged
=========== Cluster 7624 ========== 
michelin
=========== Cluster 7625 ========== 
the
=========== Cluster 7626 ========== 
beasts
=========== Cluster 7627 ========== 
retoque
=========== Cluster 7628 ========== 
allergic
=========== Cluster 7629 ========== 
like
=========== Cluster 7630 ========== 
convenience
=========== Cluster 7631 ========== 
bs
=========== Cluster 7632 ========== 
stompin
=========== Cluster 7633 ========== 
rites
=========== Cluster 7634 ========== 
alibis
=========== Cluster 7635 ========== 
aziar
=========== Cl

=========== Cluster 8360 ========== 
froo
=========== Cluster 8361 ========== 
kicks
=========== Cluster 8362 ========== 
reciprocation
=========== Cluster 8363 ========== 
cultures
=========== Cluster 8364 ========== 
timin
=========== Cluster 8365 ========== 
indside
=========== Cluster 8366 ========== 
gaby
=========== Cluster 8367 ========== 
robber
=========== Cluster 8368 ========== 
bro
=========== Cluster 8369 ========== 
sulle
=========== Cluster 8370 ========== 
hoodie
=========== Cluster 8371 ========== 
bedpost
=========== Cluster 8372 ========== 
stil
=========== Cluster 8373 ========== 
derramar
=========== Cluster 8374 ========== 
sense
=========== Cluster 8375 ========== 
shaker
=========== Cluster 8376 ========== 
poste
pathé
=========== Cluster 8377 ========== 
craziest
=========== Cluster 8378 ========== 
nicolas
=========== Cluster 8379 ========== 
aprenda
=========== Cluster 8380 ========== 
kinda
=========== Cluster 8381 ========== 
generics
=========== Cluster 83

asfixiao
observao
=========== Cluster 8940 ========== 
cotiza
=========== Cluster 8941 ========== 
fuckers
=========== Cluster 8942 ========== 
jupiter
=========== Cluster 8943 ========== 
rotate
=========== Cluster 8944 ========== 
houdini
=========== Cluster 8945 ========== 
Cheerleader
=========== Cluster 8946 ========== 
cajá
=========== Cluster 8947 ========== 
homies
=========== Cluster 8948 ========== 
refluxo
=========== Cluster 8949 ========== 
chalice
=========== Cluster 8950 ========== 
torcedores
=========== Cluster 8951 ========== 
mcdolly
=========== Cluster 8952 ========== 
unless
=========== Cluster 8953 ========== 
acrescenta
=========== Cluster 8954 ========== 
deleted
=========== Cluster 8955 ========== 
sigh
=========== Cluster 8956 ========== 
fuss
remorse
=========== Cluster 8957 ========== 
route
=========== Cluster 8958 ========== 
Forasteiro
=========== Cluster 8959 ========== 
comprehending
=========== Cluster 8960 ========== 
ending
=========== Cluster 8961 =

decretos
=========== Cluster 9594 ========== 
charisma
=========== Cluster 9595 ========== 
legacy
=========== Cluster 9596 ========== 
açaí
=========== Cluster 9597 ========== 
andy
=========== Cluster 9598 ========== 
shiver
cherish
=========== Cluster 9599 ========== 
outa
=========== Cluster 9600 ========== 
ferra
=========== Cluster 9601 ========== 
gorgeous
=========== Cluster 9602 ========== 
consequence
=========== Cluster 9603 ========== 
cleansing
=========== Cluster 9604 ========== 
realized
=========== Cluster 9605 ========== 
bryan
=========== Cluster 9606 ========== 
dare
=========== Cluster 9607 ========== 
everywhere
=========== Cluster 9608 ========== 
Wonderwall
=========== Cluster 9609 ========== 
capitais
=========== Cluster 9610 ========== 
keeping
=========== Cluster 9611 ========== 
Machine
=========== Cluster 9612 ========== 
stormwind
=========== Cluster 9613 ========== 
indiscipline
=========== Cluster 9614 ========== 
secluded
=========== Cluster 9615 =======

=========== Cluster 10226 ========== 
sprinklers
=========== Cluster 10227 ========== 
billboards
=========== Cluster 10228 ========== 
realign
=========== Cluster 10229 ========== 
untangle
=========== Cluster 10230 ========== 
spending
=========== Cluster 10231 ========== 
screwed
=========== Cluster 10232 ========== 
older
=========== Cluster 10233 ========== 
enjoy
=========== Cluster 10234 ========== 
ove
=========== Cluster 10235 ========== 
fumar
tentar
morrer
sabendo
fugir
gostando
paia
cansado
aproveito
questão
sobreviver
tardão
=========== Cluster 10236 ========== 
be
=========== Cluster 10237 ========== 
casca
=========== Cluster 10238 ========== 
leanin
=========== Cluster 10239 ========== 
hung
=========== Cluster 10240 ========== 
gonn
=========== Cluster 10241 ========== 
kiss…
=========== Cluster 10242 ========== 
vil
=========== Cluster 10243 ========== 
strager
=========== Cluster 10244 ========== 
aúnque
=========== Cluster 10245 ========== 
jabá
=========== Cluster 